## Action Plan

1) load vgg model and classes from keras

2) define paths to load the data

3) load and dump data using bcolz

4) 


In [38]:
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.utils.data_utils import get_file
import zipfile
import json
from keras.models import Sequential

In [39]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

In [40]:
# Function to download data
from __future__ import ( division, absolute_import, print_function, unicode_literals )

import sys, os, tempfile, logging

if sys.version_info >= (3,):
    import urllib.request as urllib2
    import urllib.parse as urlparse
else:
    import urllib2
    import urlparse

def download_file(url, desc=None):
    u = urllib2.urlopen(url)

    scheme, netloc, path, query, fragment = urlparse.urlsplit(url)
    filename = os.path.basename(path)
    if not filename:
        filename = 'downloaded.file'
    if desc:
        filename = os.path.join(desc, filename)

    with open(filename, 'wb') as f:
        meta = u.info()
        meta_func = meta.getheaders if hasattr(meta, 'getheaders') else meta.get_all
        meta_length = meta_func("Content-Length")
        file_size = None
        if meta_length:
            file_size = int(meta_length[0])
        print("Downloading: {0} Bytes: {1}".format(url, file_size))

        file_size_dl = 0
        block_sz = 8192
        while True:
            buffer = u.read(block_sz)
            if not buffer:
                break

            file_size_dl += len(buffer)
            f.write(buffer)

            status = "{0:16}".format(file_size_dl)
            if file_size:
                status += "   [{0:6.2f}%]".format(file_size_dl * 100 / file_size)
            status += chr(13)
            print(status, end="")
        print()
    with zipfile.ZipFile("dogscats.zip","r") as zip_ref:
        zip_ref.extractall()

    return filename

url = files_to_download = "http://www.platform.ai/data/dogscats.zip"
# filename = download_file(url)
# print(filename)

In [41]:
import json
CLASS_FILE='/home/pranand/.keras/models/imagenet_class_index.json'
def model_load(model=vgg16.VGG16(), class_path=CLASS_FILE, ):
    model = model
    classes_dict = json.load(open(class_path))
    classes = [classes_dict[str(i)][1] for i in range(len(classes_dict))]
    return model, classes


In [42]:
model, classes = model_load()

In [43]:
## Defining  a function to get data in batches from a directory
# path = '/home/pranand/courses/deeplearning1/nbs/My_Files'
path = '/home/pranand/courses/deeplearning1/nbs/My_Files/dogscats/'

batch_size=4
def get_batches(directory, gen=image.ImageDataGenerator(), shuffle=True, batch_size=batch_size, 
                class_mode='categorical', target_size=None):
    if target_size == None:
        target_size = (224, 224)
    return gen.flow_from_directory(directory, target_size=target_size, class_mode=class_mode, batch_size=batch_size
                                  , shuffle=shuffle)

In [44]:
## Append data in an array using get_data function
def get_data(batches):
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

# def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [45]:
## Loading and saving an array in python

import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [46]:
batches = get_batches(path + 'train', batch_size=batch_size)
val_batches = get_batches(path + 'valid', batch_size=batch_size)
# imgs_train,labels_train = next(batches)
# imgs_val,labels_val = next(val_batches)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [47]:
## Saving the array on disk
# train_data = get_data(batches)
# val_data = get_data(val_batches)

In [ ]:
def predict_using_vgg(batches):
    train_feature = model.predict_generator(batches, batches.nb_sample)
    return train_feature

In [ ]:
trn_features = predict_using_vgg(batches=batches)
val_features = predict_using_vgg(batches=val_batches)

In [ ]:
print(trn_features.shape)
print(val_features.shape)

In [ ]:
# Saving using bcolz

save_array('train_data.bc', trn_features)
save_array('valid_data.bc', val_features)

In [ ]:
# loading dumped data
train_data = load_array('train_data.bc')
val_data = load_array('valid_data.bc')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
def newmodel(batches, lr=0.01):
    model = Sequential()
    model.add(Dense(input_dim=1000, output_dim=batches.nb_class, activation='softmax'))
    model.compile(optimizer=Adam(lr=lr),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
batches.nb_class

In [ ]:
Newmodel = newmodel(batches)
# lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, 
#        validation_data=(val_features, val_labels))

In [ ]:
no_of_epochs = 1
for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    Newmodel.fit(train_data, trn_labels, nb_epoch=1, validation_data=(val_data, val_labels))
    latest_weights_filename = 'ft%d.h5' % epoch
    Newmodel.model.save_weights(path+latest_weights_filename)
print("Completed %s fit operations" % no_of_epochs)